In [1]:
import os
os.chdir('..')

In [13]:
from sklearn.decomposition import PCA
from matplotlib.lines import Line2D
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams, cycler
from src.utils.dataset import ms_data
from src.utils.utils import split_train_test

In [14]:
intensities_file = 'data/canis_intensities.csv'
dataset_name = intensities_file.split('/')[-1].split('_')[0]
data, labels, samples = ms_data(intensities_file)
data[np.isnan(data)] = 0

categories = pd.Categorical(labels).codes
samples_df = pd.concat([
    pd.DataFrame(np.array(samples).reshape([-1, 1])),
    pd.DataFrame(np.array(categories).reshape([-1, 1])),
    pd.DataFrame(np.array(labels).reshape([-1, 1])),
], 1)
samples_df.columns = ['sample', 'category', 'label']


In [15]:
train_samples, test_samples, train_cats = split_train_test(samples_df)
train_indices = [s for s, lab in enumerate(samples_df['sample']) if lab in train_samples]
test_indices = [s for s, lab in enumerate(samples_df['sample']) if lab in test_samples]

train_data = data[train_indices]
test_data = data[test_indices]
train_labels = samples_df['category'][train_indices].tolist()
test_labels = samples_df['category'][test_indices].tolist()


C:\ProgramData\Anaconda3\envs\AirSim\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


In [23]:
pca = PCA(n_components=2)
principal_components = pca.fit_transform(train_data, train_labels)
test_pcs = pca.transform(test_data)
principal_df = pd.DataFrame(data=principal_components, 
                            columns=['principal component 1', 'principal component 2'])
test_principal_df = pd.DataFrame(data=test_pcs, 
                                 columns=['principal component 1', 'principal component 2'])
labels_df = pd.DataFrame(train_labels)
labels_df.index = principal_df.index
final_df = pd.DataFrame(np.concatenate((principal_df.values, labels_df.values), axis=1),
                        columns=list(principal_df.columns) + ['label'])

test_labels_df = pd.DataFrame(test_labels)
test_labels_df.index = test_principal_df.index
test_final_df = pd.DataFrame(np.concatenate((test_principal_df.values, test_labels_df.values), axis=1),
                             columns=list(test_principal_df.columns) + ['label'])


In [24]:
final_df

,principal component 1,principal component 2,label
0,-33727.742188,-8197.445312,0.0
1,-34269.312500,-6819.484375,0.0
2,-35647.957031,-23493.681641,0.0
3,-35577.253906,-20799.664062,0.0
4,-36014.226562,-26555.166016,0.0
...,...,...,...
1776,-32830.667969,-21021.822266,8.0
1777,-34493.015625,-24848.757812,8.0
1778,-35368.679688,-24025.972656,8.0
1779,-35741.429688,-28022.355469,8.0


In [31]:
fig = plt.figure(figsize=(16, 12))
ax = fig.add_subplot(111)
ev = pca.explained_variance_ratio_
pc1 = 'Component_1 : ' + str(np.round(ev[0] * 100, decimals=2)) + "%"
pc2 = 'Component_2 : ' + str(np.round(ev[1] * 100, decimals=2)) + "%"
ax.set_xlabel(pc1, fontsize=15)
ax.set_ylabel(pc2, fontsize=15)
ax.set_title('2 component PCA', fontsize=20)
cmap = plt.cm.coolwarm

num_targets = len(set(train_labels))

cols = cmap(np.linspace(0, 1, num_targets))
colors = rcParams['axes.prop_cycle'] = cycler(color=cols)

colors_list = []
data1_list = []
data2_list = []
for t, target in enumerate(list(set(train_labels))):
    indices_to_keep = [True if x == target else False for x in list(final_df.label)]
    data1 = list(final_df.loc[indices_to_keep, 'principal component 1'])
    data2 = list(final_df.loc[indices_to_keep, 'principal component 2'])

    data1_list += [data1]
    data2_list += [data2]
    colors_list += [np.array([[cols[t]] * len(data1)])]

colors_vector = np.hstack(colors_list).squeeze()
data1_vector = np.hstack(data1_list).reshape(-1, 1)
data2_vector = np.hstack(data2_list).reshape(-1, 1)
data_colors_vector = np.concatenate((data1_vector, data2_vector, colors_vector), axis=1)
data1 = data_colors_vector[:, 0]
data2 = data_colors_vector[:, 1]
col = data_colors_vector[:, 2:]

ax.scatter(data1, data2, s=20, alpha=0.5, c=col, label=train_labels)
custom_lines = [Line2D([0], [0], color=cols[x], lw=4) for x in range(num_targets)]

colors_list = []
data1_list = []
data2_list = []
for t, target in enumerate(list(set(test_labels))):
    indices_to_keep = [True if x == target else False for x in list(test_final_df.label)]
    data1 = list(test_final_df.loc[indices_to_keep, 'principal component 1'])
    data2 = list(test_final_df.loc[indices_to_keep, 'principal component 2'])

    data1_list += [data1]
    data2_list += [data2]
    colors_list += [np.array([[cols[t]] * len(data1)])]

colors_vector = np.hstack(colors_list).squeeze()
data1_vector = np.hstack(data1_list).reshape(-1, 1)
data2_vector = np.hstack(data2_list).reshape(-1, 1)
data_colors_vector = np.concatenate((data1_vector, data2_vector, colors_vector), axis=1)
data1 = data_colors_vector[:, 0]
data2 = data_colors_vector[:, 1]
col = data_colors_vector[:, 2:]

ax.scatter(data1, data2, s=30, alpha=0.5, c=col, label=test_labels, marker='x')

custom_lines += [Line2D([0], [0], color=cols[x], lw=4) for x in range(num_targets)]

ax.legend(custom_lines, list(set(samples_df['label'])))

plt.show()
plt.close()


In [20]:
sum(indices_to_keep)

0